In [8]:
!pip install scikit-image

    100% |████████████████████████████████| 25.3MB 2.7MB/s eta 0:00:01
    100% |████████████████████████████████| 31.2MB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 675kB 38.4MB/s ta 0:00:01
    100% |████████████████████████████████| 4.4MB 16.0MB/s ta 0:00:01
    100% |████████████████████████████████| 1.7MB 28.4MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 31.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [63]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
get_label = {
    'articulated_truck': 0,
    'background': 1,
    'bicycle': 2,
    'bus': 3,
    'car': 4,
    'motorcycle': 5,
    'non-motorized_vehicle': 6,
    'pedestrian': 7,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

validation_split = .2
shuffle_dataset = True
random_seed= 42
num_epochs = 2
num_classes = 11
batch_size = 100
learning_rate = 0.001

In [64]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
    
    def __call__(self, sample):
        image = sample['image']
        h, w = image.shape[:2]
        
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        
        new_h, new_w = int(new_h), int(new_w)
        
        img = transform.resize(image, (new_h, new_w))
        return {'image': img, 'label': sample['label']}
class ToTensor(object):

    def __call__(self, sample):
        image = sample['image']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'label': sample['label']}

In [65]:
class DS(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.transform = transform
        self.csv_ds = pd.read_csv(csv_file, dtype='str')
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.csv_ds)
    
    def __getitem__(self, idx):
#         print(self.csv_ds.iloc[idx][0], self.csv_ds.iloc[idx][1])
        img_name = str(self.csv_ds.iloc[idx][0])
        img_path = os.path.join(self.root_dir, 'train',str(self.csv_ds.iloc[idx][1]), str(self.csv_ds.iloc[idx][0]) + '.jpg')
        image = io.imread(img_path)
        label = get_label[self.csv_ds.iloc[idx][1]]
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        return sample
    
tds = DS(csv_file='./gt_train.csv', root_dir='./',
                                    transform = transforms.Compose([Rescale((100, 100)), ToTensor()]) )
dataset_size = len(tds)

indices = list(range(dataset_size))

split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(tds, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(tds, batch_size=batch_size,

                                                
                                                sampler=validation_sampler)

# dl = torch.utils.data.DataLoader(dataset=tds, batch_size=10, shuffle=True)

# model = CNNModel()
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)



In [66]:



class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(20000, num_classes+1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [68]:

model = ConvNet(num_classes)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    



    print(len(train_loader))
    for i, x in enumerate(train_loader):
        images = Variable(x['image'])
        labels = Variable(x['label'])
#         print(images.shape, labels)
        
        outputs = model(images.float())
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         if i == 10:
#             break
            
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


4154
Epoch [1/2], Step [100/4154], Loss: 0.7855
Epoch [1/2], Step [200/4154], Loss: 0.7625
Epoch [1/2], Step [300/4154], Loss: 0.6630
Epoch [1/2], Step [400/4154], Loss: 0.4832
Epoch [1/2], Step [500/4154], Loss: 0.4994
Epoch [1/2], Step [600/4154], Loss: 0.5163
Epoch [1/2], Step [700/4154], Loss: 0.4016
Epoch [1/2], Step [800/4154], Loss: 0.4643
Epoch [1/2], Step [900/4154], Loss: 0.5043
Epoch [1/2], Step [1000/4154], Loss: 0.2977
Epoch [1/2], Step [1100/4154], Loss: 0.3598
Epoch [1/2], Step [1200/4154], Loss: 0.2509
Epoch [1/2], Step [1300/4154], Loss: 0.3706
Epoch [1/2], Step [1400/4154], Loss: 0.5253
Epoch [1/2], Step [1500/4154], Loss: 0.3112
Epoch [1/2], Step [1600/4154], Loss: 0.4270
Epoch [1/2], Step [1700/4154], Loss: 0.4334
Epoch [1/2], Step [1800/4154], Loss: 0.4352
Epoch [1/2], Step [1900/4154], Loss: 0.4108
Epoch [1/2], Step [2000/4154], Loss: 0.5355
Epoch [1/2], Step [2100/4154], Loss: 0.3222
Epoch [1/2], Step [2200/4154], Loss: 0.4537
Epoch [1/2], Step [2300/4154], Loss:

In [79]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for i, x in enumerate(validation_loader):
#         print(x)
        images = x['image']
#         labels_ = []
#         for label in labels:
#             labels_.append(int(label))
#         labels = torch.LongTensor(labels)
        labels = x['label']
#         print(images, labels)
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(labels.size(0), (predicted == labels).sum().item())


    print('Test Accuracy of the model on the ~51057 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

100 83
100 92
100 89
100 85
100 87
100 89
100 87
100 92
100 82
100 88
100 84
100 85
100 83
100 92
100 88
100 90
100 92
100 95
100 88
100 89
100 84
100 88
100 87
100 81
100 90
100 89
100 78
100 93
100 90
100 87
100 87
100 77
100 83
100 85
100 90
100 88
100 85
100 89
100 84
100 81
100 90
100 86
100 87
100 89
100 88
100 83
100 88
100 89
100 94
100 90
100 88
100 88
100 89
100 86
100 92
100 88
100 82
100 88
100 85
100 85
100 86
100 83
100 86
100 89
100 90
100 86
100 90
100 84
100 91
100 89
100 91
100 89
100 85
100 82
100 80
100 89
100 87
100 89
100 91
100 88
100 91
100 87
100 90
100 91
100 85
100 83
100 89
100 83
100 89
100 90
100 91
100 75
100 81
100 89
100 87
100 87
100 86
100 96
100 91
100 86
100 87
100 88
100 91
100 87
100 83
100 83
100 88
100 86
100 86
100 90
100 89
100 86
100 89
100 87
100 84
100 84
100 88
100 87
100 86
100 86
100 88
100 89
100 87
100 87
100 87
100 91
100 88
100 89
100 89
100 91
100 87
100 90
100 88
100 91
100 83
100 83
100 86
100 89
100 92
100 86
100 92
100 88
100 82

ValueError: axes don't match array